In [3]:
# !pip install pandas

In [1]:
import json
import random
import pandas as pd

In [85]:
name = "ml-100k"

inter_path = f"../../../data/resource/{name}/{name}.inter"
user_path = f"../../../data/resource/{name}/{name}.user"
item_path = f"../../../data/resource/{name}/{name}.item"

w_pt_path = f"./data/llm/{name}/{name}_pt.json"
w_ft_path = f"./data/llm/{name}/{name}_ft.json"
w_pd_path = f"./data/llm/{name}/{name}_pd.json"

In [87]:
# 读数据
inter_data = pd.read_csv(inter_path, sep="\t").sort_values(by=['user_id:token','rating:float', 'timestamp:float'], ascending=[True, False, True])
user_data = pd.read_csv(user_path, sep="\t").sort_values(by=['user_id:token'], ascending=[True])
item_data = pd.read_csv(item_path, sep="\t").sort_values(by=['item_id:token'], ascending=[True])
item_data.loc[item_data['item_id:token'] == 1,'class:token_seq']

0    Animation Children's Comedy
Name: class:token_seq, dtype: object

In [89]:
user_data

,user_id:token,age:token,gender:token,occupation:token,zip_code:token
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [91]:
category_mapping = {
    'Action': ['Action'],
    'Action Adventure': ['Action', 'Adventure'],
    "Action Adventure Animation Children's Fantasy": ['Action', 'Adventure', "Animation", "Children's", "Fantasy"],
    'Action Adventure Animation Horror Sci-Fi': ['Action', 'Adventure', 'Animation', 'Horror', 'Sci-Fi'],
    "Action Adventure Children's": ['Action', 'Adventure', "Children's"],
    "Action Adventure Children's Fantasy": ['Action', 'Adventure', "Children's", "Fantasy"],
    "Action Adventure Children's Fantasy Sci-Fi": ['Action', 'Adventure', "Children's", "Fantasy", 'Sci-Fi'],
    "Action Adventure Children's Sci-Fi": ['Action', 'Adventure', "Children's", 'Sci-Fi'],
    'Action Adventure Comedy': ['Action', 'Adventure', 'Comedy'],
    'Action Adventure Comedy Crime': ['Action', 'Adventure', 'Comedy', 'Crime'],
    'Action Adventure Comedy Horror': ['Action', 'Adventure', 'Comedy', 'Horror'],
    'Action Adventure Comedy Horror Sci-Fi': ['Action', 'Adventure', 'Comedy', 'Horror', 'Sci-Fi'],
    'Action Adventure Comedy Musical Thriller': ['Action', 'Adventure', 'Comedy', 'Musical', 'Thriller'],
    'Action Adventure Comedy Romance': ['Action', 'Adventure', 'Comedy', 'Romance'],
    'Action Adventure Comedy Sci-Fi': ['Action', 'Adventure', 'Comedy', 'Sci-Fi'],
    'Action Adventure Comedy War': ['Action', 'Adventure', 'Comedy', 'War'],
    'Action Adventure Crime': ['Action', 'Adventure', 'Crime'],
    'Action Adventure Crime Drama': ['Action', 'Adventure', 'Crime', 'Drama'],
    'Action Adventure Crime Thriller': ['Action', 'Adventure', 'Crime', 'Thriller'],
    'Action Adventure Drama': ['Action', 'Adventure', 'Drama'],
    'Action Adventure Drama Romance': ['Action', 'Adventure', 'Drama', 'Romance'],
    'Action Adventure Drama Romance Sci-Fi War': ['Action', 'Adventure', 'Drama', 'Romance', 'Sci-Fi', 'War'],
    'Action Adventure Fantasy': ['Action', 'Adventure', 'Fantasy'],
    'Action Adventure Mystery': ['Action', 'Adventure', 'Mystery'],
    'Action Adventure Mystery Sci-Fi': ['Action', 'Adventure', 'Mystery', 'Sci-Fi'],
    'Action Adventure Romance': ['Action', 'Adventure', 'Romance'],
    'Action Adventure Romance Sci-Fi War': ['Action', 'Adventure', 'Romance', 'Sci-Fi', 'War'],
    'Action Adventure Romance War': ['Action', 'Adventure', 'Romance', 'War'],
    'Action Adventure Sci-Fi': ['Action', 'Adventure', 'Sci-Fi'],
    'Action Adventure Sci-Fi Thriller': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'],
    'Action Adventure Sci-Fi War': ['Action', 'Adventure', 'Sci-Fi', 'War'],
    'Action Adventure Thriller': ['Action', 'Adventure', 'Thriller'],
    'Action Adventure Western': ['Action', 'Adventure', 'Western'],
    "Action Animation Children's Sci-Fi Thriller War": ['Action', 'Animation', "Children's", 'Sci-Fi', 'Thriller', 'War'],
    "Action Children's": ['Action', "Children's"],
    'Action Comedy': ['Action', 'Comedy'],
    'Action Comedy Crime Drama': ['Action', 'Comedy', 'Crime', 'Drama'],
    'Action Comedy Crime Horror Thriller': ['Action', 'Comedy', 'Crime', 'Horror', 'Thriller'],
    'Action Comedy Drama': ['Action', 'Comedy', 'Drama'],
    'Action Comedy Musical': ['Action', 'Comedy', 'Musical'],
    'Action Comedy Musical Sci-Fi': ['Action', 'Comedy', 'Musical', 'Sci-Fi'],
    'Action Comedy Sci-Fi War': ['Action', 'Comedy', 'Sci-Fi', 'War'],
    'Action Comedy War': ['Action', 'Comedy', 'War'],
    'Action Comedy Western': ['Action', 'Comedy', 'Western'],
    'Action Crime': ['Action', 'Crime'],
    'Action Crime Drama': ['Action', 'Crime', 'Drama'],
    'Action Crime Mystery': ['Action', 'Crime', 'Mystery'],
    'Action Crime Romance': ['Action', 'Crime', 'Romance'],
    'Action Crime Sci-Fi': ['Action', 'Crime', 'Sci-Fi'],
    'Action Crime Thriller': ['Action', 'Crime', 'Thriller'],
    'Action Drama': ['Action', 'Drama'],
    'Action Drama Mystery': ['Action', 'Drama', 'Mystery'],
    'Action Drama Mystery Romance Thriller': ['Action', 'Drama', 'Mystery', 'Romance', 'Thriller'],
    'Action Drama Romance': ['Action', 'Drama', 'Romance'],
    'Action Drama Romance War': ['Action', 'Drama', 'Romance', 'War'],
    'Action Drama Thriller': ['Action', 'Drama', 'Thriller'],
    'Action Drama Thriller War': ['Action', 'Drama', 'Thriller', 'War'],
    'Action Drama War': ['Action', 'Drama', 'War'],
    'Action Drama Western': ['Action', 'Drama', 'Western'],
    'Action Horror': ['Action', 'Horror'],
    'Action Horror Sci-Fi': ['Action', 'Horror', 'Sci-Fi'],
    'Action Horror Sci-Fi Thriller': ['Action', 'Horror', 'Sci-Fi', 'Thriller'],
    'Action Mystery Romance Thriller': ['Action', 'Mystery', 'Romance', 'Thriller'],
    'Action Mystery Sci-Fi Thriller': ['Action', 'Mystery', 'Sci-Fi', 'Thriller'],
    'Action Mystery Thriller': ['Action', 'Mystery', 'Thriller'],
    'Action Romance': ['Action', 'Romance'],
    'Action Romance Thriller': ['Action', 'Romance', 'Thriller'],
    'Action Romance War': ['Action', 'Romance', 'War'],
    'Action Sci-Fi': ['Action', 'Sci-Fi'],
    'Action Sci-Fi Thriller': ['Action', 'Sci-Fi', 'Thriller'],
    'Action Sci-Fi Thriller War': ['Action', 'Sci-Fi', 'Thriller', 'War'],
    'Action Sci-Fi War': ['Action', 'Sci-Fi', 'War'],
    'Action Thriller': ['Action', 'Thriller'],
    'Action Thriller War': ['Action', 'Thriller', 'War'],
    'Action Western': ['Action', 'Western'],
    'Adventure': ['Adventure'],
    "Adventure Animation Children's Comedy Fantasy": ['Adventure', 'Animation', "Children's", 'Comedy', 'Fantasy'],
    "Adventure Animation Children's Comedy Musical": ['Adventure', 'Animation', "Children's", 'Comedy', 'Musical'],
    "Adventure Animation Children's Musical": ['Adventure', 'Animation', "Children's", 'Musical'],
    'Adventure Animation Sci-Fi Thriller': ['Adventure', 'Animation', 'Sci-Fi', 'Thriller'],
    "Adventure Children's": ['Adventure', "Children's"],
    "Adventure Children's Comedy": ['Adventure', "Children's", 'Comedy'],
    "Adventure Children's Comedy Fantasy Romance Sci-Fi": ['Adventure', "Children's", 'Comedy', 'Fantasy', 'Romance', 'Sci-Fi'],
    "Adventure Children's Drama": ['Adventure', "Children's", 'Drama'],
    "Adventure Children's Drama Musical": ['Adventure', "Children's", 'Drama', 'Musical'],
    "Adventure Children's Fantasy": ['Adventure', "Children's", 'Fantasy'],
    "Adventure Children's Fantasy Sci-Fi": ['Adventure', "Children's", 'Fantasy', 'Sci-Fi'],
    "Adventure Children's Musical": ['Adventure', "Children's", 'Musical'],
    "Adventure Children's Romance": ['Adventure', "Children's", 'Romance'],
    'Adventure Comedy': ['Adventure', 'Comedy'],
    'Adventure Comedy Drama': ['Adventure', 'Comedy', 'Drama'],
    'Adventure Drama': ['Adventure', 'Drama'],
    'Adventure Drama Western': ['Adventure', 'Drama', 'Western'],
    'Adventure Romance': ['Adventure', 'Romance'],
    'Adventure Sci-Fi': ['Adventure', 'Sci-Fi'],
    'Adventure Sci-Fi Thriller': ['Adventure', 'Sci-Fi', 'Thriller'],
    'Adventure Thriller': ['Adventure', 'Thriller'],
    'Adventure War': ['Adventure', 'War'],
    'Animation': ['Animation'],
    "Animation Children's": ['Animation', "Children's"],
    "Animation Children's Comedy": ['Animation', "Children's", 'Comedy'],
    "Animation Children's Comedy Musical": ['Animation', "Children's", 'Comedy', 'Musical'],
    "Animation Children's Comedy Romance": ['Animation', "Children's", 'Comedy', 'Romance'],
    "Animation Children's Musical": ['Animation', "Children's", 'Musical'],
    "Animation Children's Musical Romance": ['Animation', "Children's", 'Musical', 'Romance'],
    'Animation Comedy': ['Animation', 'Comedy'],
    'Animation Comedy Thriller': ['Animation', 'Comedy', 'Thriller'],
    'Animation Sci-Fi': ['Animation', 'Sci-Fi'],
    "Children's": ["Children's"],
    "Children's Comedy": ["Children's", 'Comedy'],
    "Children's Comedy Drama": ["Children's", 'Comedy', 'Drama'],
    "Children's Comedy Fantasy": ["Children's", 'Comedy', 'Fantasy'],
    "Children's Comedy Musical": ["Children's", 'Comedy', 'Musical'],
    "Children's Comedy Mystery": ["Children's", 'Comedy', 'Mystery'],
    "Children's Comedy Western": ["Children's", 'Comedy', 'Western'],
    "Children's Drama": ["Children's", 'Drama'],
    "Children's Drama Fantasy": ["Children's", 'Drama', 'Fantasy'],
    "Children's Drama Fantasy Sci-Fi": ["Children's", 'Drama', 'Fantasy', 'Sci-Fi'],
    "Children's Fantasy": ["Children's", 'Fantasy'],
    'Comedy': ['Comedy'],
    'Comedy Crime': ['Comedy', 'Crime'],
    'Comedy Crime Drama': ['Comedy', 'Crime', 'Drama'],
    'Comedy Crime Drama Mystery': ['Comedy', 'Crime', 'Drama', 'Mystery'],
    'Comedy Crime Fantasy': ['Comedy', 'Crime', 'Fantasy'],
    'Comedy Crime Horror': ['Comedy', 'Crime', 'Horror'],
    'Comedy Crime Mystery Thriller': ['Comedy', 'Crime', 'Mystery', 'Thriller'],
    'Comedy Documentary': ['Comedy', 'Documentary'],
    'Comedy Drama': ['Comedy', 'Drama'],
    'Comedy Drama Musical': ['Comedy', 'Drama', 'Musical'],
    'Comedy Drama Romance': ['Comedy', 'Drama', 'Romance'],
    'Comedy Drama Thriller': ['Comedy', 'Drama', 'Thriller'],
    'Comedy Drama War': ['Comedy', 'Drama', 'War'],
    'Comedy Fantasy': ['Comedy', 'Fantasy'],
    'Comedy Fantasy Romance Sci-Fi': ['Comedy', 'Fantasy', 'Romance', 'Sci-Fi'],
    'Comedy Horror': ['Comedy', 'Horror'],
    'Comedy Musical': ['Comedy', 'Musical'],
    'Comedy Musical Romance': ['Comedy', 'Musical', 'Romance'],
    'Comedy Mystery': ['Comedy', 'Mystery'],
    'Comedy Mystery Romance': ['Comedy', 'Mystery', 'Romance'],
    'Comedy Mystery Romance Thriller': ['Comedy', 'Mystery', 'Romance', 'Thriller'],
    'Comedy Mystery Thriller': ['Comedy', 'Mystery', 'Thriller'],
    'Comedy Romance': ['Comedy', 'Romance'],
    'Comedy Romance Thriller': ['Comedy', 'Romance', 'Thriller'],
    'Comedy Romance War': ['Comedy', 'Romance', 'War'],
    'Comedy Sci-Fi': ['Comedy', 'Sci-Fi'],
    'Comedy Thriller': ['Comedy', 'Thriller'],
    'Comedy War': ['Comedy', 'War'],
    'Comedy Western': ['Comedy', 'Western'],
    'Crime': ['Crime'],
    'Crime Drama': ['Crime', 'Drama'],
    'Crime Drama Film-Noir': ['Crime', 'Drama', 'Film-Noir'],
    'Crime Drama Mystery': ['Crime', 'Drama', 'Mystery'],
    'Crime Drama Mystery Thriller': ['Crime', 'Drama', 'Mystery', 'Thriller'],
    'Crime Drama Romance': ['Crime', 'Drama', 'Romance'],
    'Crime Drama Romance Thriller': ['Crime', 'Drama', 'Romance', 'Thriller'],
    'Crime Drama Sci-Fi': ['Crime', 'Drama', 'Sci-Fi'],
    'Crime Drama Thriller': ['Crime', 'Drama', 'Thriller'],
    'Crime Film-Noir': ['Crime', 'Film-Noir'],
    'Crime Film-Noir Mystery': ['Crime', 'Film-Noir', 'Mystery'],
    'Crime Film-Noir Mystery Thriller': ['Crime', 'Film-Noir', 'Mystery', 'Thriller'],
    'Crime Film-Noir Thriller': ['Crime', 'Film-Noir', 'Thriller'],
    'Crime Horror Mystery Thriller': ['Crime', 'Horror', 'Mystery', 'Thriller'],
    'Crime Thriller': ['Crime', 'Thriller'],
    'Documentary': ['Documentary'],
    'Documentary Drama': ['Documentary', 'Drama'],
    'Documentary War': ['Documentary', 'War'],
    'Drama': ['Drama'],
    'Drama Fantasy Thriller': ['Drama', 'Fantasy', 'Thriller'],
    'Drama Horror': ['Drama', 'Horror'],
    'Drama Musical': ['Drama', 'Musical'],
    'Drama Musical War': ['Drama', 'Musical', 'War'],
    'Drama Mystery': ['Drama', 'Mystery'],
    'Drama Mystery Romance': ['Drama', 'Mystery', 'Romance'],
    'Drama Mystery Sci-Fi Thriller': ['Drama', 'Mystery', 'Sci-Fi', 'Thriller'],
    'Drama Mystery Thriller': ['Drama', 'Mystery', 'Thriller'],
    'Drama Romance': ['Drama', 'Romance'],
    'Drama Romance Thriller': ['Drama', 'Romance', 'Thriller'],
    'Drama Romance War': ['Drama', 'Romance', 'War'],
    'Drama Romance War Western': ['Drama', 'Romance', 'War', 'Western'],
    'Drama Sci-Fi': ['Drama', 'Sci-Fi'],
    'Drama Sci-Fi Thriller': ['Drama', 'Sci-Fi', 'Thriller'],
    'Drama Thriller': ['Drama', 'Thriller'],
    'Drama Thriller War': ['Drama', 'Thriller', 'War'],
    'Drama War': ['Drama', 'War'],
    'Drama Western': ['Drama', 'Western'],
    'Fantasy': ['Fantasy'],
    'Film-Noir': ['Film-Noir'],
    'Film-Noir Mystery': ['Film-Noir', 'Mystery'],
    'Film-Noir Mystery Thriller': ['Film-Noir', 'Mystery', 'Thriller'],
    'Film-Noir Romance Thriller': ['Film-Noir', 'Romance', 'Thriller'],
    'Film-Noir Sci-Fi': ['Film-Noir', 'Sci-Fi'],
    'Film-Noir Sci-Fi Thriller': ['Film-Noir', 'Sci-Fi', 'Thriller'],
    'Film-Noir Thriller': ['Film-Noir', 'Thriller'],
    'Horror': ['Horror'],
    'Horror Mystery Thriller': ['Horror', 'Mystery', 'Thriller'],
    'Horror Romance': ['Horror', 'Romance'],
    'Horror Romance Thriller': ['Horror', 'Romance', 'Thriller'],
    'Horror Sci-Fi': ['Horror', 'Sci-Fi'],
    'Horror Sci-Fi Thriller': ['Horror', 'Sci-Fi', 'Thriller'],
    'Horror Thriller': ['Horror', 'Thriller'],
    'Musical': ['Musical'],
    'Musical Romance': ['Musical', 'Romance'],
    'Mystery': ['Mystery'],
    'Mystery Romance Thriller': ['Mystery', 'Romance', 'Thriller'],
    'Mystery Sci-Fi': ['Mystery', 'Sci-Fi'],
    'Mystery Thriller': ['Mystery', 'Thriller'],
    'Romance': ['Romance'],
    'Romance Thriller': ['Romance', 'Thriller'],
    'Romance War': ['Romance', 'War'],
    'Sci-Fi': ['Sci-Fi'],
    'Sci-Fi Thriller': ['Sci-Fi', 'Thriller'],
    'Sci-Fi War': ['Sci-Fi', 'War'],
    'Thriller': ['Thriller'],
    'War': ['War'],
    'Western': ['Western'],
    'unknown': ['unknown']
}

In [93]:
category_mapping['Mystery Sci-Fi']

['Mystery', 'Sci-Fi']

In [138]:
threshold = 3
max_length = 10
item_temp = "Item id: item_{item_id}, movie title: {movie_title}, release year: {release_year}, category: {class}, liking_rate: {liking_rate}."
# user_temp = "User id: user_{user_id}, age: {age}, gender: {gender}, occupation: {occupation}, zipcode: {zip_code}, View: {most_viewed_genres}, Like: {best_rated_genres}."
user_temp = "User id: user_{user_id}, age: {age}, gender: {gender}, occupation: {occupation}, zipcode: {zip_code}, Top View: {most_viewed_genres}, Like: {best_rated_genres}."

# # inter_temp = "The user user_{user_id} likes the following movies: {likes} and dislikes the following movies: {dislikes}."
user_train_temp = """You are a recommendation system that gives you a list of movies that the user likes and dislikes, and predicts whether the user will like the target movie or not. Returns only whether the user likes the target movie: Yes or No. # noqa: E501

User Information:
{user_info}

Likes List:
{likes_list}

Dislikes List:
{dislikes_list}

Target Movie:
{targe_movie}
"""

In [97]:
# 产品评价
def build_item_other(row):
    df1 = inter_data[inter_data['item_id:token'] == row['item_id:token']]
    total = df1.shape[0]
    likes_count = df1[df1['rating:float'] > threshold].shape[0]
    # dislikes_count = total - likes_count
    liking_rate = round(likes_count/total, 4)
    return {'liking_rate':liking_rate}

In [99]:
item_data['other'] = item_data.apply(lambda row: build_item_other(row), axis=1)
item_data['other'].iloc[0]


{'liking_rate': 0.7102}

In [101]:
item_data['category'] = item_data.apply(lambda row: category_mapping[row['class:token_seq']],axis = 1)
item_data

,item_id:token,movie_title:token_seq,release_year:token,class:token_seq,other,category
0,1,Toy Story,1995,Animation Children's Comedy,{'liking_rate': 0.7102},"[Animation, Children's, Comedy]"
1,2,GoldenEye,1995,Action Adventure Thriller,{'liking_rate': 0.3893},"[Action, Adventure, Thriller]"
2,3,Four Rooms,1995,Thriller,{'liking_rate': 0.3778},[Thriller]
3,4,Get Shorty,1995,Action Comedy Drama,{'liking_rate': 0.5837},"[Action, Comedy, Drama]"
4,5,Copycat,1995,Crime Drama Thriller,{'liking_rate': 0.4535},"[Crime, Drama, Thriller]"
...,...,...,...,...,...,...
1677,1678,Mat' i syn,1997,Drama,{'liking_rate': 0.0},[Drama]
1678,1679,B. Monkey,1998,Romance Thriller,{'liking_rate': 0.0},"[Romance, Thriller]"
1679,1680,Sliding Doors,1998,Drama Romance,{'liking_rate': 0.0},"[Drama, Romance]"
1680,1681,You So Crazy,1994,Comedy,{'liking_rate': 0.0},[Comedy]


In [103]:
# 拼接信息, json
item_data['info'] = item_data.apply(lambda row: {'text':item_temp.replace('{movie_title}',row["movie_title:token_seq"]).replace('{item_id}',str(row["item_id:token"])).replace('{release_year}',str(row["release_year:token"])).replace('{class}',', '.join(map(str, row["category"]))).replace('{liking_rate}',str(row['other']["liking_rate"]))}, axis=1)

In [105]:
item_data

,item_id:token,movie_title:token_seq,release_year:token,class:token_seq,other,category,info
0,1,Toy Story,1995,Animation Children's Comedy,{'liking_rate': 0.7102},"[Animation, Children's, Comedy]","{'text': 'Item id: item_1, movie title: Toy St..."
1,2,GoldenEye,1995,Action Adventure Thriller,{'liking_rate': 0.3893},"[Action, Adventure, Thriller]","{'text': 'Item id: item_2, movie title: Golden..."
2,3,Four Rooms,1995,Thriller,{'liking_rate': 0.3778},[Thriller],"{'text': 'Item id: item_3, movie title: Four R..."
3,4,Get Shorty,1995,Action Comedy Drama,{'liking_rate': 0.5837},"[Action, Comedy, Drama]","{'text': 'Item id: item_4, movie title: Get Sh..."
4,5,Copycat,1995,Crime Drama Thriller,{'liking_rate': 0.4535},"[Crime, Drama, Thriller]","{'text': 'Item id: item_5, movie title: Copyca..."
...,...,...,...,...,...,...,...
1677,1678,Mat' i syn,1997,Drama,{'liking_rate': 0.0},[Drama],"{'text': 'Item id: item_1678, movie title: Mat..."
1678,1679,B. Monkey,1998,Romance Thriller,{'liking_rate': 0.0},"[Romance, Thriller]","{'text': 'Item id: item_1679, movie title: B. ..."
1679,1680,Sliding Doors,1998,Drama Romance,{'liking_rate': 0.0},"[Drama, Romance]","{'text': 'Item id: item_1680, movie title: Sli..."
1680,1681,You So Crazy,1994,Comedy,{'liking_rate': 0.0},[Comedy],"{'text': 'Item id: item_1681, movie title: You..."


In [107]:
# 分组&聚合
data_new = inter_data
data_u = data_new.groupby(['user_id:token']).count()
data_u["count"] = data_u["item_id:token"]
# data_u = data_u.drop(['item_id:token','num_repeat:float','timestamp:float'], axis=1)

# 筛选=>10 core
data_u = data_u[data_u["count"] >= 10]

# 按user_id筛选
inter_data = data_new[data_new["user_id:token"].isin(data_u.index)]
# 按user_id分组，并将其他字段存为列表
inter_data = inter_data.groupby('user_id:token').agg(list).reset_index()
inter_data

,user_id:token,item_id:token,rating:float,timestamp:float
0,1,"[168, 172, 165, 196, 166, 14, 127, 181, 109, 1...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[874965478, 874965478, 874965518, 874965677, 8..."
1,2,"[275, 251, 50, 313, 242, 283, 311, 100, 127, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, ...","[888550939, 888552084, 888552084, 888552084, 8..."
2,3,"[328, 321, 347, 340, 346, 320, 344, 342, 331, ...","[5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...","[889237455, 889237455, 889237455, 889237455, 8..."
3,4,"[258, 300, 303, 329, 327, 359, 362, 360, 361, ...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[892001374, 892001445, 892002352, 892002352, 8..."
4,5,"[257, 100, 109, 181, 89, 228, 172, 174, 396, 4...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[875635239, 875635349, 875635350, 875635757, 8..."
...,...,...,...,...
938,939,"[689, 326, 127, 9, 257, 1190, 742, 222, 591, 2...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[880260636, 880260636, 880260745, 880260745, 8..."
939,940,"[300, 258, 313, 482, 508, 96, 427, 709, 8, 56,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, ...","[884801316, 884801316, 884801316, 885921198, 8..."
940,941,"[117, 408, 181, 298, 919, 124, 1, 258, 300, 29...","[5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, ...","[875048886, 875048886, 875048887, 875048887, 8..."
941,942,"[347, 272, 304, 300, 500, 50, 282, 528, 71, 99...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[891282396, 891282420, 891282457, 891282564, 8..."


In [116]:
# 用户使用的产品类型列表
# inter_data.iloc[0]
# inter_data.iloc[0]['item_id:token']
# 将嵌套列表展开为一个平面列表
# category_list = [category for item_id in inter_data.iloc[0]['item_id:token'] for category in category_mapping[item_data.loc[item_data['item_id:token'] == item_id,'class:token_seq'].values[0]]]
# df = pd.DataFrame(category_list, columns=['category'])
# 统计各个值出现的次数
#counts = df['category'].value_counts().reset_index()
# print(counts)


# 添加新列：user各个种类item的观看数量
def max_view_genres(row):
    df1 = row
    category_list = [category for item_id in row['item_id:token'] for category in category_mapping[item_data.loc[item_data['item_id:token'] == item_id,'class:token_seq'].values[0]]]
    df = pd.DataFrame(category_list, columns=['category'])
    count = df['category'].value_counts().reset_index().to_dict('records')
    res = {item['category']: item['count'] for item in count}
    return res

def max_view_genres_top(row):
    df1 = row
    category_list = [category for item_id in row['item_id:token'] for category in category_mapping[item_data.loc[item_data['item_id:token'] == item_id,'class:token_seq'].values[0]]]
    df = pd.DataFrame(category_list, columns=['category'])
    count = df['category'].value_counts().head(5).reset_index().to_dict('records')
    res = {item['category']: item['count'] for item in count}
    return res

In [118]:
user_data['most_viewed_genres:object'] = inter_data.apply(lambda row: max_view_genres(row), axis = 1)
user_data['most_viewed_genres_top:object'] = inter_data.apply(lambda row: max_view_genres_top(row), axis = 1)
user_data

,user_id:token,age:token,gender:token,occupation:token,zip_code:token,most_viewed_genres:object,most_viewed_genres_top:object
0,1,24,M,technician,85711,"{'Drama': 107, 'Comedy': 91, 'Action': 75, 'Th...","{'Drama': 107, 'Comedy': 91, 'Action': 75, 'Th..."
1,2,53,F,other,94043,"{'Drama': 35, 'Romance': 16, 'Comedy': 16, 'Th...","{'Drama': 35, 'Romance': 16, 'Comedy': 16, 'Th..."
2,3,23,M,writer,32067,"{'Drama': 22, 'Thriller': 21, 'Action': 14, 'C...","{'Drama': 22, 'Thriller': 21, 'Action': 14, 'C..."
3,4,24,M,technician,43537,"{'Thriller': 11, 'Action': 8, 'Drama': 6, 'Sci...","{'Thriller': 11, 'Action': 8, 'Drama': 6, 'Sci..."
4,5,33,F,other,15213,"{'Comedy': 82, 'Action': 56, 'Sci-Fi': 33, 'Ad...","{'Comedy': 82, 'Action': 56, 'Sci-Fi': 33, 'Ad..."
...,...,...,...,...,...,...,...
938,939,26,F,student,33319,"{'Action': 18, 'Drama': 18, 'Comedy': 15, 'Thr...","{'Action': 18, 'Drama': 18, 'Comedy': 15, 'Thr..."
939,940,32,M,administrator,02215,"{'Drama': 48, 'Comedy': 42, 'Romance': 24, 'Ac...","{'Drama': 48, 'Comedy': 42, 'Romance': 24, 'Ac..."
940,941,20,M,student,97229,"{'Action': 10, 'Sci-Fi': 8, 'Adventure': 7, 'T...","{'Action': 10, 'Sci-Fi': 8, 'Adventure': 7, 'T..."
941,942,48,F,librarian,78209,"{'Drama': 31, 'Comedy': 22, 'Thriller': 20, 'A...","{'Drama': 31, 'Comedy': 22, 'Thriller': 20, 'A..."


In [120]:
def count_item_rating(row):
    df1 = row
    likes_count = len([x for x in df1['rating:float'] if x > threshold])
    category_list = [category for item_id in row['item_id:token'][0:likes_count] for category in category_mapping[item_data.loc[item_data['item_id:token'] == item_id,'class:token_seq'].values[0]]]
    df = pd.DataFrame(category_list, columns=['category'])
    count = df['category'].value_counts().head(5).reset_index().to_dict('records')
    res = {item['category']: item['count'] for item in count}
    return res
    

In [122]:
user_data['best_rated_genres:object'] = inter_data.apply(lambda row: count_item_rating(row), axis = 1)

In [123]:
user_data.loc[0]['best_rated_genres:object']

{'Drama': 78, 'Comedy': 49, 'Action': 39, 'Sci-Fi': 32, 'Thriller': 30}

In [126]:
def cal_arr(ls,num):
    arr = []
    for i in range(0, len(ls), num):
        a1 = ls[i:i+num]
        if len(a1)==num:
            arr.append(a1)
    return arr
def build_user_inter(row):
    rs = row['rating:float']
    p_len = len([x for x in rs if x > threshold])
    p_rs = rs[:p_len]
    n_rs = rs[p_len:]

    ids = row['item_id:token']
    p_id_groups = ids[:p_len]
    n_id_groups = ids[p_len:]

    # 6p,4n
    p_num,n_num = 6,4
    p_rs1 = cal_arr(p_rs,p_num)
    n_rs1 = cal_arr(n_rs,n_num)

    p_id_groups1 = cal_arr(p_id_groups,p_num)
    n_id_groups1 = cal_arr(n_id_groups,n_num)
    
    # 对齐两个列表
    min_num = min(len(p_rs1), len(n_rs1))
    p_rs = p_rs1[:min_num]
    n_rs = n_rs1[:min_num]

    p_id_groups = p_id_groups1[:min_num]
    n_id_groups = n_id_groups1[:min_num]

    inter = {'p_id_groups': p_id_groups, 'n_id_groups': n_id_groups, 'p_rs': p_rs, 'n_rs': n_rs}
    return inter

inter_data['user_inter'] = inter_data.apply(lambda row: build_user_inter(row), axis=1)
inter_data.iloc[0]

user_id:token                                                      1
item_id:token      [168, 172, 165, 196, 166, 14, 127, 181, 109, 1...
rating:float       [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...
timestamp:float    [874965478, 874965478, 874965518, 874965677, 8...
user_inter         {'p_id_groups': [[168, 172, 165, 196, 166, 14]...
Name: 0, dtype: object

In [128]:
def esteem(row):
    res =  {key: row['best_rated_genres:object'][key] / row['most_viewed_genres:object'][key] for key in row['best_rated_genres:object']} 
    return res

In [130]:
user_data['esteem'] = user_data.apply(lambda row: esteem(row), axis = 1)

In [132]:
user_data['esteem']

0      {'Drama': 0.7289719626168224, 'Comedy': 0.5384...
1      {'Drama': 0.6571428571428571, 'Romance': 0.812...
2      {'Drama': 0.36363636363636365, 'Thriller': 0.2...
3      {'Thriller': 0.6363636363636364, 'Drama': 0.83...
4      {'Comedy': 0.4024390243902439, 'Action': 0.410...
                             ...                        
938    {'Drama': 0.8888888888888888, 'Comedy': 0.9333...
939    {'Drama': 0.5625, 'Comedy': 0.5476190476190477...
940    {'Action': 0.7, 'Comedy': 0.8571428571428571, ...
941    {'Drama': 0.9032258064516129, 'Comedy': 0.8181...
942    {'Action': 0.65625, 'Drama': 0.701754385964912...
Name: esteem, Length: 943, dtype: object

In [140]:
# 拼接信息, json
# user_data['info'] = user_data.apply(lambda row: {'text':user_temp.replace('{user_id}',str(row["user_id:token"])).replace('{age}',str(row["age:token"])).replace('{gender}',str(row["gender:token"])).replace('{occupation}',row["occupation:token"]).replace('{zip_code}',str(row["zip_code:token"])).replace('{most_viewed_genres}',str(row["most_viewed_genres:object"])).replace('{best_rated_genres}',str(row["best_rated_genres:object"]))}, axis=1)
#user_data['info'] = user_data.apply(lambda row: {'text':user_temp.replace('{user_id}',str(row["user_id:token"])).replace('{age}',str(row["age:token"])).replace('{gender}',str(row["gender:token"])).replace('{occupation}',row["occupation:token"]).replace('{zip_code}',str(row["zip_code:token"])).replace('{most_viewed_genres}',str(row["most_viewed_genres:object"])).replace('{best_rated_genres}',str(row["esteem"]))}, axis=1)
user_data['info'] = user_data.apply(lambda row: {'text':user_temp.replace('{user_id}',str(row["user_id:token"])).replace('{age}',str(row["age:token"])).replace('{gender}',str(row["gender:token"])).replace('{occupation}',row["occupation:token"]).replace('{zip_code}',str(row["zip_code:token"])).replace('{most_viewed_genres}',str(row["most_viewed_genres_top:object"])).replace('{best_rated_genres}',str(row["esteem"]))}, axis=1)

user_data['info'].iloc[0]

{'text': "User id: user_1, age: 24, gender: M, occupation: technician, zipcode: 85711, Top View: {'Drama': 107, 'Comedy': 91, 'Action': 75, 'Thriller': 52, 'Romance': 44}, Like: {'Drama': 0.7289719626168224, 'Comedy': 0.5384615384615384, 'Action': 0.52, 'Sci-Fi': 0.7441860465116279, 'Thriller': 0.5769230769230769}."}

In [142]:
def cal_label(rating):
    return "Yes" if rating > threshold else "No"

In [144]:
def build_item_infos(ids):
    item_infos = ''
    for i in ids:
        # item_row = item_data[item_data["item_id:token"]==i].iloc[0]
        item_infos += f'item id: item_{i} \n'
    return item_infos

def build_inter(row):
    #拼接用户信息
    user_row = user_data[user_data["user_id:token"]==row["user_id:token"]].iloc[0]
    # user_info = f'user id: user_{user_row["user_id:token"]}, age: {user_row["age:token"]}, gender: {user_row["gender:token"]}, occupation: {user_row["occupation:token"]}, zipcode: {user_row["zip_code:token"]}, View: {user_row["most_viewed_genres:object"]}, Like: {user_row["best_rated_genres:object"]}'
    user_info = f'user id: user_{user_row["user_id:token"]}, age: {user_row["age:token"]}, gender: {user_row["gender:token"]}, occupation: {user_row["occupation:token"]}, zipcode: {user_row["zip_code:token"]}, View: {user_row["most_viewed_genres:object"]}, Like: {user_row["esteem"]}'
    
    #细分输入数据，按照6：4比率划分，且随机取出一个数据作为target
    user_inter = row['user_inter']
    p_id_groups = user_inter['p_id_groups']
    n_id_groups = user_inter['n_id_groups']
    if random.random() < 0.8:
        add = 'ft'
        ft_user_list.append(row['user_id:token'])
    else:
        add = 'pd'
        pd_user_list.append(row['user_id:token'])
    for p_ids,n_ids in zip(p_id_groups,n_id_groups):
        # [0,1)根据0.6和0.4的比例进行线性转换
        if random.random() < 0.6:
            iid = p_ids.pop()
            flag = 'Yes'
        else:
            iid = n_ids.pop()
            flag = 'No'
        target_item_row = item_data[item_data["item_id:token"]==iid].iloc[0]
        target_movie = f'item id: item_{iid}'
        #拼接prompt
        user_temp = user_train_temp.replace('{user_info}', user_info).replace('{targe_movie}',target_movie).replace('{likes_list}', build_item_infos(p_ids)).replace('{dislikes_list}', build_item_infos(n_ids)).replace('\n\n\n','\n\n')
        user_dict = {"instruction": user_temp, "input": '', "output": flag}
        if add == 'ft':
            ft_list.append(user_dict)
        else:
            pd_list.append(user_dict)

In [146]:
# 数据分组   微调、预测组，预测组用户从微调组中取出
ft_list = [] #微调
pd_list = [] #预测
ft_user_list = [] #微调用户列表
pd_user_list = [] #预测用户列表
for idx, row in inter_data.iterrows():
    build_inter(row)

In [148]:
print(len(ft_user_list))
print(len(ft_list))
print(len(pd_user_list))
print(len(pd_list))

764
5590
179
1283


In [152]:
ft_list[10]

{'instruction': 'You are a recommendation system that gives you a list of movies that the user likes and dislikes, and predicts whether the user will like the target movie or not. Returns only whether the user likes the target movie: Yes or No. # noqa: E501\n\nUser Information:\nuser id: user_1, age: 24, gender: M, occupation: technician, zipcode: 85711, View: {\'Drama\': 107, \'Comedy\': 91, \'Action\': 75, \'Thriller\': 52, \'Romance\': 44, \'Sci-Fi\': 43, \'Adventure\': 42, "Children\'s": 25, \'Crime\': 25, \'War\': 25, \'Horror\': 13, \'Musical\': 13, \'Animation\': 12, \'Western\': 6, \'Mystery\': 5, \'Documentary\': 5, \'Fantasy\': 2, \'Film-Noir\': 1, \'unknown\': 1}, Like: {\'Drama\': 0.7289719626168224, \'Comedy\': 0.5384615384615384, \'Action\': 0.52, \'Sci-Fi\': 0.7441860465116279, \'Thriller\': 0.5769230769230769}\n\nLikes List:\nitem id: item_12 \nitem id: item_208 \nitem id: item_228 \nitem id: item_44 \nitem id: item_86 \n\nDislikes List:\nitem id: item_49 \nitem id: ite

In [154]:
len(pd_list)

1283

In [156]:
len(ft_list)

5590

In [160]:
# list类型数据转化为json
def export_json(data,export_path):
    df = pd.DataFrame(data)
    # 导出为格式化后的 JSON 文件
    df.to_json(export_path, orient='records')

In [162]:
user_data[user_data['user_id:token'].isin(ft_user_list)]['info']

0      {'text': 'User id: user_1, age: 24, gender: M,...
1      {'text': 'User id: user_2, age: 53, gender: F,...
2      {'text': 'User id: user_3, age: 23, gender: M,...
3      {'text': 'User id: user_4, age: 24, gender: M,...
4      {'text': 'User id: user_5, age: 33, gender: F,...
                             ...                        
937    {'text': 'User id: user_938, age: 38, gender: ...
938    {'text': 'User id: user_939, age: 26, gender: ...
940    {'text': 'User id: user_941, age: 20, gender: ...
941    {'text': 'User id: user_942, age: 48, gender: ...
942    {'text': 'User id: user_943, age: 22, gender: ...
Name: info, Length: 764, dtype: object

In [164]:
user_data['info']

0      {'text': 'User id: user_1, age: 24, gender: M,...
1      {'text': 'User id: user_2, age: 53, gender: F,...
2      {'text': 'User id: user_3, age: 23, gender: M,...
3      {'text': 'User id: user_4, age: 24, gender: M,...
4      {'text': 'User id: user_5, age: 33, gender: F,...
                             ...                        
938    {'text': 'User id: user_939, age: 26, gender: ...
939    {'text': 'User id: user_940, age: 32, gender: ...
940    {'text': 'User id: user_941, age: 20, gender: ...
941    {'text': 'User id: user_942, age: 48, gender: ...
942    {'text': 'User id: user_943, age: 22, gender: ...
Name: info, Length: 943, dtype: object

In [166]:
#调用写入文件中
export_json(item_data['info'].tolist()+user_data['info'].tolist(),w_pt_path)
export_json(pd_list,w_pd_path)
export_json(ft_list,w_ft_path)

In [168]:
# 读取 JSON 文件到 DataFrame
text_df = pd.read_json(w_pt_path)
text_df

,text
0,"Item id: item_1, movie title: Toy Story, relea..."
1,"Item id: item_2, movie title: GoldenEye, relea..."
2,"Item id: item_3, movie title: Four Rooms, rele..."
3,"Item id: item_4, movie title: Get Shorty, rele..."
4,"Item id: item_5, movie title: Copycat, release..."
...,...
2620,"User id: user_939, age: 26, gender: F, occupat..."
2621,"User id: user_940, age: 32, gender: M, occupat..."
2622,"User id: user_941, age: 20, gender: M, occupat..."
2623,"User id: user_942, age: 48, gender: F, occupat..."
